In [3]:
import duckdb
import polars as pl

from utils.id_generation import generate_ids


## Database Instantiation and ID Generation

In [2]:
conn = duckdb.connect(r'database/mealpass.duckdb')

# Drop the table if it already exists
conn.execute("""DROP TABLE IF EXISTS MealPass;""")

# Create the table
conn.execute("""
CREATE TABLE MealPass (
    MealPassID TEXT PRIMARY KEY,
    isRedeemed BOOLEAN DEFAULT FALSE
);
""")
conn.commit()
conn.close()

In [5]:
meal_passes = generate_ids(prefix='MP_', quantity=2002)
data_insertion = []
for meal_pass in meal_passes:
#     encryption_key = generate_random_encryption_key()
#     encrypted_id = encrypt_message(meal_pass, encryption_key)

    data_insertion.append((
#         encrypted_id,
#         encryption_key,
        meal_pass,
    ))

data_insertion

[('MP_00001',),
 ('MP_00002',),
 ('MP_00003',),
 ('MP_00004',),
 ('MP_00005',),
 ('MP_00006',),
 ('MP_00007',),
 ('MP_00008',),
 ('MP_00009',),
 ('MP_00010',),
 ('MP_00011',),
 ('MP_00012',),
 ('MP_00013',),
 ('MP_00014',),
 ('MP_00015',),
 ('MP_00016',),
 ('MP_00017',),
 ('MP_00018',),
 ('MP_00019',),
 ('MP_00020',),
 ('MP_00021',),
 ('MP_00022',),
 ('MP_00023',),
 ('MP_00024',),
 ('MP_00025',),
 ('MP_00026',),
 ('MP_00027',),
 ('MP_00028',),
 ('MP_00029',),
 ('MP_00030',),
 ('MP_00031',),
 ('MP_00032',),
 ('MP_00033',),
 ('MP_00034',),
 ('MP_00035',),
 ('MP_00036',),
 ('MP_00037',),
 ('MP_00038',),
 ('MP_00039',),
 ('MP_00040',),
 ('MP_00041',),
 ('MP_00042',),
 ('MP_00043',),
 ('MP_00044',),
 ('MP_00045',),
 ('MP_00046',),
 ('MP_00047',),
 ('MP_00048',),
 ('MP_00049',),
 ('MP_00050',),
 ('MP_00051',),
 ('MP_00052',),
 ('MP_00053',),
 ('MP_00054',),
 ('MP_00055',),
 ('MP_00056',),
 ('MP_00057',),
 ('MP_00058',),
 ('MP_00059',),
 ('MP_00060',),
 ('MP_00061',),
 ('MP_00062',),
 ('MP_00

In [4]:
data_insertion = pl.DataFrame(
    {
        "MealPassID": list(generate_ids(prefix='MP_', quantity=2002)),
        "isRedeemed": False
    }
)

# Connect to DuckDB database
conn = duckdb.connect(r"database/mealpass.duckdb")

# Insert the data using executemany
conn.executemany("""
INSERT INTO MealPass (MealPassID, isRedeemed) VALUES (?, ?)
""", data_insertion.iter_rows())

# Close the connection
conn.commit()
conn.close()

In [34]:
conn = duckdb.connect(r"database/mealpass.duckdb")
data = conn.execute("SELECT * FROM MealPass").pl()
conn.close()
data

MealPassID,isRedeemed
str,bool
"""MP_00001""",false
"""MP_00002""",false
"""MP_00003""",false
"""MP_00004""",false
"""MP_00005""",false
…,…
"""MP_01998""",false
"""MP_01999""",false
"""MP_02000""",false


In [39]:
# Saving to different formats
data.write_csv('database/meal_passes.csv')
data.write_parquet('database/meal_passes.parquet', compression='snappy')
data.write_json('database/meal_passes.json')

## QR Code Generation

In [5]:
import os
import shutil

import qrcode
from tqdm import tqdm

In [7]:
conn = duckdb.connect(r"database/mealpass.duckdb")
data = conn.execute("SELECT * FROM MealPass").pl()
conn.close()
data

MealPassID,isRedeemed
str,bool
"""MP_00001""",false
"""MP_00002""",false
"""MP_00003""",false
"""MP_00004""",false
"""MP_00005""",false
…,…
"""MP_01998""",false
"""MP_01999""",false
"""MP_02000""",false


In [17]:
try:
    if os.path.exists('qr_codes'):
        shutil.rmtree('qr_codes')

    os.makedirs('qr_codes')

    for meal_pass in tqdm(data.iter_rows(), desc="Generating QR codes", total=len(data)):
        meal_pass_id = meal_pass[0]
        # Generate the QR code
        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,
        )

        qr.add_data(meal_pass_id)
        qr.make(fit=True)

        # Create and save the QR code image
        img = qr.make_image(fill='black', back_color='white')
        img.save(f"qr_codes/{meal_pass_id}.png")

except Exception as e:
    print(f"{type(e).__name__}: Error in generating QR codes. {e}")


Generating QR codes: 100%|██████████| 2002/2002 [00:08<00:00, 238.77it/s]


## PDF Mass Generation

In [4]:
from typing import List
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

In [26]:
class MealStubPDF(canvas.Canvas):
    """A class to create a PDF with meal stub details, including QR code, ID, and border.
    Methods
    -------
    draw_qr_code(x, y, image_path, width, height)
        Draws a QR code image at the specified location with the given dimensions.
    draw_id(text)
        Draws the ID text at the specified location.
    draw_border(x, y, width, height)
        Draws a border rectangle at the specified location with the given dimensions.
    """

    def draw_qr_code(self, x, y, image_path, width, height):
        self.drawImage(image_path, x, y, width, height)

    def draw_border(self, x, y, width, height, dashed):

        if dashed:
            self.setDash(1, 10)
        self.rect(x, y, width, height)

    def draw_admit_line(self, start_x, start_y, end_x, end_y, dashed):
        if dashed:
            self.setDash(2, 3)

        self.line(start_x, start_y, end_x, end_y)

def cm_to_points(centimeter) -> float:
    """Convert a measurement from centimeters to points.

    Args:
        centimeter (float): The measurement in centimeters to be converted.

    Returns:
        float: The measurement converted to points.
    """
    return centimeter * 72 / 2.54

def inch_to_points(inch) -> float:
    """Convert a measurement from inches to points.

    Args:
        inch (float): The measurement in inches to be converted.

    Returns:
        float: The measurement converted to points.
    """

    return inch * 72

def create_A4_batch(
        ids: List[str],
        pdf = MealStubPDF
        ):
    """ Creates a batch of meal stub PDFs in A4 paper size.

    Args:
        ids (List[str]): List of IDs to generate QR codes and meal stubs for.
        pdf (class, optional): PDF generation class. Defaults to MealStubPDF.

    Raises:
        Exception: If there is an error in creating the PDF.

    The function performs the following steps:
        1. Sets up the save directory and filename for the PDF.
        2. Registers the custom font "Night-Driver".
        3. Defines the page size as A4.
        4. Initializes the PDF with the specified filename and page size.
        5. Draws a middle border line on the PDF.
        6. Iterates over the list of IDs to generate QR codes and meal stubs on both the left and right sides of the page.
        7. Draws QR codes, borders, admit lines, brand names, and admit strings for each ID.
        8. Saves the PDF after all stubs are generated.
    """

    try:
        save_directory = "pdfs"
        filename = f"{ids[0]}-{ids[-1]}.pdf"
        full_path = os.path.join(save_directory, filename)
        os.makedirs(save_directory, exist_ok=True)

        font_filename = "nightdriver.ttf"
        font_path = os.path.join("static/fonts", font_filename)
        font_name = "Night-Driver"
        pdfmetrics.registerFont(TTFont(font_name, font_path))

        pagesize = (
            inch_to_points(8.3),
            inch_to_points(13.5) # legal paper size
            ) # A4 Paper size

        meal_stub = pdf(
            filename = full_path,
            pagesize = pagesize # A4 Paper size
            )

        meal_stub.line(
            inch_to_points(4.15),
            inch_to_points(0),
            inch_to_points(4.15),
            inch_to_points(14)
            ) # Middle Border

        box_width = inch_to_points(4.15 - 0.50)
        box_height = inch_to_points(1)

        ids_left = ids[0:13]
        ids_right = ids[13:26]

        for i in range(0, len(ids_left), 1):

            # <Left Side>
            image_directory = "qr_codes"
            image_file_left = f"{ids_left[i]}.png"
            image_path_left = os.path.join(image_directory, image_file_left)

            meal_stub.draw_qr_code(
                x = inch_to_points(0.25),
                y = inch_to_points(i + 0.25),
                image_path = image_path_left,
                width = cm_to_points(2.5),
                height = cm_to_points(2.5)
                )

            meal_stub.draw_border(
                x = inch_to_points(0.25), # Left margin
                y = inch_to_points(i + 0.25), # Bottom margin
                width = box_width,
                height = box_height,
                dashed = True
                )

            meal_stub.draw_admit_line(
                start_x=box_width - inch_to_points(0.25),
                start_y=inch_to_points(i + 0.25),
                end_x=box_width - inch_to_points(0.25),
                end_y=inch_to_points(i + 1.25),
                dashed=True
            )

            # Brand Name
            meal_stub.setFillColor(colors.black)
            meal_stub.setFont("Times-Bold", 12)
            meal_stub.drawString(
                x = inch_to_points(2.1),
                y = inch_to_points(i + 0.95),
                text = "MCES"
            )

            meal_stub.setFillColor(colors.red)
            meal_stub.setFont("Night-Driver", 15)
            meal_stub.drawString(
                x = inch_to_points(1.73),
                y = inch_to_points(i + 0.65),
                text = "BALIK-LANTAW"
            )

            meal_stub.setFillColor(colors.red)
            meal_stub.setFont("Night-Driver", 12)
            meal_stub.drawString(
                x = inch_to_points(2.15),
                y = inch_to_points(i + 0.4),
                text = "2026"
            )

            meal_stub.saveState()
            x = inch_to_points(3.7)
            y = inch_to_points(i + 0.25 + 0.15)
            meal_stub.translate(x, y)
            meal_stub.rotate(90)

            # Admit String
            meal_stub.setFillColor(colors.black)
            meal_stub.setFont("Helvetica", 9)
            meal_stub.drawString(
                x = inch_to_points(-0.055),
                y = inch_to_points(0),
                text = "1 MEAL STUB"
            )

            meal_stub.setFillColor(colors.black)
            meal_stub.drawString(
                x = inch_to_points(0.05),
                y = inch_to_points(2.4),
                text = ids_left[i]
            )

            meal_stub.restoreState()

            # <Right Side>
            image_file_right = f"{ids_right[i]}.png"
            image_path_right = os.path.join(image_directory, image_file_right)

            meal_stub.draw_qr_code(
                x = inch_to_points(4.15 + .25),
                y = inch_to_points(i + 0.25),
                image_path = image_path_right,
                width = cm_to_points(2.5),
                height = cm_to_points(2.5)
                )

            meal_stub.draw_border(
                x = inch_to_points(4.15 + 0.25), # Left margin
                y = inch_to_points(i + 0.25), # Bottom margin
                width = box_width,
                height = box_height,
                dashed = True
            )

            meal_stub.draw_admit_line(
                start_x=inch_to_points(8.3 - 0.75),
                start_y=inch_to_points(i + 0.25),
                end_x=inch_to_points(8.3 - 0.75),
                end_y=inch_to_points(i + 1.25),
                dashed=True
            )

            meal_stub.saveState()
            x = inch_to_points((8.3 / 2) + 3.7)
            y = inch_to_points(i + 0.25 + 0.15)
            meal_stub.translate(x, y)
            meal_stub.rotate(90)

            # Admit String
            meal_stub.setFillColor(colors.black)
            meal_stub.setFont("Helvetica", 9)
            meal_stub.drawString(
                x = inch_to_points(-0.055),
                y = inch_to_points(0),
                text = "1 MEAL STUB"
            )

            meal_stub.setFillColor(colors.black)
            meal_stub.drawString(
                x = inch_to_points(0.05),
                y = inch_to_points(2.4),
                text = ids_right[i]
            )
            meal_stub.restoreState()

            # Brand Name
            meal_stub.setFillColor(colors.black)
            meal_stub.setFont("Times-Bold", 12)
            meal_stub.drawString(
                x = inch_to_points((8.3 / 2) + 2.1),
                y = inch_to_points(i + 0.95),
                text = "MCES"
            )
            meal_stub.setFillColor(colors.red)
            meal_stub.setFont("Night-Driver", 15)
            meal_stub.drawString(
                x = inch_to_points((8.3 / 2) + 1.73),
                y = inch_to_points(i + 0.65),
                text = "BALIK-LANTAW"
            )
            meal_stub.setFillColor(colors.red)
            meal_stub.setFont("Night-Driver", 12)
            meal_stub.drawString(
                x = inch_to_points((8.3 / 2) + 2.15),
                y = inch_to_points(i + 0.4),
                text = "2026"
            )
    except Exception as e:
        print(f"{type(e).__name__}: Error in creating PDF. {e}")
    else:
        meal_stub.save()

In [31]:
try:
    if os.path.exists('pdfs'):
        shutil.rmtree('pdfs')

    os.makedirs('pdfs')

    N = 26
    # Loop through the DataFrame in chunks of n rows
    for i in tqdm(range(0, len(data), N), desc="Generating PDF files"):
        chunk = data[i:i+N]['MealPassID']
    
        ids = chunk.to_numpy()

        create_A4_batch(ids)
except Exception as e:
    print(f"{type(e).__name__}: Error in generating PDF files. {e}")

Generating PDF files: 100%|██████████| 77/77 [00:17<00:00,  4.40it/s]


## Zipping the files

In [40]:
from utils.batch_processing import zip_batch

zip_batch(['qr_codes', 'pdfs', 'database'])
print("Done.")

Zipping folders: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]

batch-2025-01-09 09-01-40.zip: Zipping completed successfully.
Done.
